In [6]:
import pandas as pd

df = pd.read_csv('./products-01.csv')

df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   brand           10 non-null     object
 1   movement_type   10 non-null     object
 2   gender          10 non-null     object
 3   availability    10 non-null     object
 4   watch_style     10 non-null     object
 5   material        10 non-null     object
 6   season          10 non-null     object
 7   launch_year     10 non-null     int64 
 8   adjective       10 non-null     object
 9   design_keyword  10 non-null     object
 10  price           10 non-null     int64 
 11  description     10 non-null     object
dtypes: int64(2), object(10)
memory usage: 1.1+ KB


In [42]:
from agents import Agent, function_tool, Runner, RunContextWrapper, trace, handoff, RunConfig
from pydantic import BaseModel, Field
from openai import OpenAI
from openai.types.responses import (
    ResponseCreatedEvent,
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseFunctionCallArgumentsDoneEvent,
    ResponseFunctionToolCall,
    ResponseInProgressEvent,
    ResponseOutputItemAddedEvent,
    ResponseOutputMessage,
    ResponseTextDeltaEvent,
    ResponseContentPartDoneEvent
)
import strip_markdown as smd
import json

df = pd.read_csv('./products-01.csv')


In [8]:
df.head()

,brand,movement_type,gender,availability,watch_style,material,season,launch_year,adjective,design_keyword,price,description
0,Tag Heuer,Quartz,Women,Out of Stock,Skeleton,Platinum,Adventure,2011,Vanguard,Pulse,4166,This is a Tag Heuer Skeleton watch with a Quar...
1,Tag Heuer,Manual,Women,Out of Stock,Aviator,Platinum,Formal Occasions,2015,Phantom,Voyager,5501,This is a Tag Heuer Aviator watch with a Manua...
2,Tissot,Automatic,Women,Limited Edition,Diver,Stainless Steel,Summer,2009,Aurora,Titan,1970,This is a Tissot Diver watch with a Automatic ...
3,Rolex,Automatic,Men,Limited Edition,Field,Stainless Steel,Winter,1981,Vanguard,Legacy,4275,This is a Rolex Field watch with a Automatic m...
4,Omega,Manual,Women,Limited Edition,Aviator,Platinum,Formal Occasions,2009,Pristine,Chronos,7163,This is a Omega Aviator watch with a Manual mo...


In [9]:
# Find watches of branch Omega  
df[(df['brand'] == 'Omega') & (df['price'] > 5000)]

,brand,movement_type,gender,availability,watch_style,material,season,launch_year,adjective,design_keyword,price,description
4,Omega,Manual,Women,Limited Edition,Aviator,Platinum,Formal Occasions,2009,Pristine,Chronos,7163,This is a Omega Aviator watch with a Manual mo...
6,Omega,Smart Hybrid,Unisex,Pre-Order,Diver,Ceramic,Formal Occasions,1983,Stealth,Aero,9895,This is a Omega Diver watch with a Smart Hybri...


In [10]:
df[(df['brand'] == 'Omega') & (df['price'] > 6000)].head()

,brand,movement_type,gender,availability,watch_style,material,season,launch_year,adjective,design_keyword,price,description
4,Omega,Manual,Women,Limited Edition,Aviator,Platinum,Formal Occasions,2009,Pristine,Chronos,7163,This is a Omega Aviator watch with a Manual mo...
6,Omega,Smart Hybrid,Unisex,Pre-Order,Diver,Ceramic,Formal Occasions,1983,Stealth,Aero,9895,This is a Omega Diver watch with a Smart Hybri...


In [4]:
df[(df['brand'] == 'Omega') & (df['price'] > 6000)].head()

,brand,movement_type,gender,availability,watch_style,material,season,launch_year,adjective,design_keyword,price,description
0,Omega,Quartz,Men,Limited Edition,Field,Gold,All Season,2022,Aurora,Pulse,6311,This is a Omega Field watch with a Quartz move...
1,Omega,Smart Hybrid,Women,Out of Stock,Diver,Gold,Adventure,1980,Eternal,Titan,7075,This is a Omega Diver watch with a Smart Hybri...
14,Omega,Automatic,Women,In Stock,Chronograph,Stainless Steel,Formal Occasions,1998,Majestic,Horizon,6379,This is a Omega Chronograph watch with a Autom...
15,Omega,Manual,Women,In Stock,Skeleton,Stainless Steel,All Season,1998,Aurora,Legacy,8530,This is a Omega Skeleton watch with a Manual m...
23,Omega,Quartz,Men,Out of Stock,Diver,Titanium,Adventure,2001,Eternal,Legacy,9710,This is a Omega Diver watch with a Quartz move...


In [30]:
def data_query_agent_handoff(ctx: RunContextWrapper[None]):
    print('Handing off to data query agent')

def execute_query_agent_handoff(ctx: RunContextWrapper[None]):
    print('Handing off to execute query agent')

def refiner_query_agent_handoff(ctx: RunContextWrapper[None]):
    print('Handing off to execute query refiner agent')

def generator_query_agent_handoff(ctx: RunContextWrapper[None]):
    print('Handing off to execute query generator agent')

In [15]:
class QueryGeneratorOutput(BaseModel):
    query: str = Field(description="A valid pandas expression that can be used to query the data.")

In [43]:
# a function that takes a pandas expression and executes it and returns the result, if result is an error, return the error message
def execute_query(query: str):
    try:
        result = eval(query, {'df': df})
        if isinstance(result, pd.Series):
            result = result.to_dict()
        elif isinstance(result, pd.DataFrame):
            result = result.to_dict(orient='records')
        return json.dumps({"results": result} if result else {"error": "No products found matching your criteria"})
    except Exception as e:
        return str(e)

In [44]:
final_result=execute_query("df[df['brand'] == 'Omega']")
smd.strip_markdown(final_result)


'{"results": [{"brand": "Omega", "movement_type": "Manual", "gender": "Women", "availability": "Limited Edition", "watch_style": "Aviator", "material": "Platinum", "season": "Formal Occasions", "launch_year": 2009, "adjective": "Pristine", "design_keyword": "Chronos", "price": 7163, "description": "This is a Omega Aviator watch with a Manual movement. It is Pristine and Chronos. It is made of Platinum and is available in Formal Occasions. It was launched in 2009. It is Limited Edition."}, {"brand": "Omega", "movement_type": "Smart Hybrid", "gender": "Unisex", "availability": "Pre-Order", "watch_style": "Diver", "material": "Ceramic", "season": "Formal Occasions", "launch_year": 1983, "adjective": "Stealth", "design_keyword": "Aero", "price": 9895, "description": "This is a Omega Diver watch with a Smart Hybrid movement. It is Stealth and Aero. It is made of Ceramic and is available in Formal Occasions. It was launched in 1983. It is Pre-Order."}]}'

In [38]:
query_execution_agent = Agent(
    name="query_execution_agent_v1",
    model="gpt-4o-mini",
    instructions=(
        "You are a helpful assistant that executes a pandas expression using the `execute_query` tool and return the results."
        "You should return the results in a human readable format. In a concise and convincing way."
    ),
    tools=[execute_query]
)

# # Query Generator Agent
query_generator_agent = Agent(
    name="query_generator_agent_v1",
    model="gpt-4o-mini",
    instructions=(
        "You are a helpful assistant that can answer questions related to business data. "
        "The data is available in a pandas DataFrame named 'df'. "
        "You must always use 'df' as the DataFrame variable in your pandas expressions. "
        f"Seeing the refined query, {df.columns} and {df.head()}, you generate a pandas expression to query the data."
    ),
    output_type=QueryGeneratorOutput,
    # handoffs=[
    #     handoff(query_execution_agent,on_handoff=execute_query_agent_handoff)
    # ]
)

# Query Refiner Agent
query_refiner_agent = Agent(
    name="query_refiner_agent_v1",
    model="gpt-4o-mini",
    instructions=(
        "You are a helpful assistant that can refine queries to be more specific and accurate."
        "If user asks for best seller or for random suggestions, search for perfumes with maximum rating."
        "You receive a query and generate a refined query in plain english that another agent will use to generate a pandas expression."
        "The query should be concise and to the point, and should not include any instructions or explanations."
    ),
    handoffs=[handoff(query_generator_agent,on_handoff=generator_query_agent_handoff)],
)


data_query_agent = Agent(
    name="data_query_agent_v1",
    handoffs=[
        handoff(query_refiner_agent,on_handoff=refiner_query_agent_handoff)
        ],
)

root_agent = Agent(
    name="Root Agent",
    instructions=(
      "You are a friendly and talkative assistant representing WatchCompany, a premium watch brand known for its exquisite collection of watches. "
        "Introduce yourself as WatchCompany's brand representative and share our passion for creating unique, high-quality watches. "
        "Engage users warmly, answer general questions, and encourage them to ask about our watches, collections, or anything related to the WatchCompany brand. "
        "If the user asks about watches and products and list or some information, delegate the query to `data_query_agent`. "    ),
    model="gpt-4o-mini",
    handoffs=[
        handoff(data_query_agent,on_handoff=data_query_agent_handoff)
        ]
)


In [45]:
customer_query="Find watches of branch Omega"
result=await Runner.run(root_agent, customer_query)
print(result.final_output)

Handing off to data query agent
Handing off to execute query refiner agent
Find high-rated watches by the brand Omega.
